# QIE Implementaion

This code implement a QIE approch named algorithm 2.1 from the paper A. Mandelbaum and S. Zeltyn, “Estimating characteristics of queueingnetworks  using  transactional  data", Queueing systems,  vol.  29,  no.  1,pp. 75–127, 1998.

Packages in use

In [79]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from datetime import datetime
import pickle
import os

Functions

In [80]:
def build_index_continuity(df , max_enqueue):
    df_1 = df[df['service'] <= max_enqueue]
    df_1 = df_1.sort_values(by=['service']) 
    df_1 = df_1.reset_index()
    z = []
    for i in range(len(df_1)):
        z.append(df_1.at[i, 'service'])
    if max_enqueue != df_1.at[len(df_1)-1, 'service']:
        z.append(max_enqueue)
        flag = 1
    else:
        flag = 0   
    minim = z[0]
    z = list(map(lambda x: x - minim, z))
    return z, flag

In [81]:
def build_ex_times(df , max_enqueue):
    minim = df['service'].min()
    df_1 = df[df['service'] > max_enqueue]
    df_1 = df_1.sort_values(by=['service']) 
    df_1 = df_1.reset_index()
    z = []
    for i in range(len(df_1)):
        z.append(df_1.at[i, 'service'])
    z = list(map(lambda x: x - minim, z))
    return z

In [82]:
# The buliding based on index that start from 0 to customers
# Build a list of tupels (left= index g_i, right= possible admissible_states)
def build_admissible_states(m, l):
    G = []
    G.append((0,[0]))
    for i in range(1,m):
        num_list = []
        for j in range(i,l):
            num_list.append(j)
        G.append((i,num_list))
    return G
            

In [83]:
# Build one transtion matrix between two djacent admissible states 
def build_transition_matrix(current_g, current_zi, former_g, z, arrival_rate, upper_boundary_condition, flag):
    p = [] # initiation
    if flag == 0: #Regular transition
        x = arrival_rate*(z[current_zi]-z[current_zi-1])
        value = math.exp(-x)
        calac_dic = {
            0: value 
        }
        
        for i in range(1, (current_g[len(current_g)-1] - former_g[0]) + 1 ):
            value = value*(x/i)
            calac_dic[i] = value
            
        for i in range(len(former_g)):
            k_i = former_g[i] #row index of the matrix
            temp = []
            for j in range(len(current_g)):
                k_j = current_g[j] # column index of the matrix 
                if k_i <= k_j: # enqueues are accumulate -> no option to decrease 
                    px = calac_dic[k_j-k_i]
                    temp.append(px)        
                else: 
                    temp.append(0)
            p.append(temp)
            
    else: # transtion matrix between t_m to t_m+1 (flag==1)
        x = arrival_rate*(z[current_zi]-z[current_zi-1])
        value = math.exp(-x)
        temp = []
        k_j = current_g[0] # column index of the matrix
        k_i = former_g[len(former_g)-1]
        p.append([value])
        pp = 1 #delete after it work
        for i in range(len(former_g)-2,-1,-1): # reverse run to reduce calculations
            k_i = former_g[i] #row index of the matrix
            value = (value*x)/(pp)
            p.append([value])
            pp += 1
        p.reverse()
              
    P = np.matrix(p) #convert to Numpy matrix
    return P 


In [84]:
# Place all the transition matrices in one list
def build_transition_matrices(G, z, m, upper_boundary_condition, arrival_rate, max_list ):
  
    #Transtion between admissible states - return P (list) of Numpy matrices
    P = []
    list_num = 0
    j = 0 
    for i in range(len(G)-1): 
        p = build_transition_matrix(G[i+1][1], G[i+1][0],
                                    G[i][1], z, arrival_rate, upper_boundary_condition, flag = 0)
        P.append(p)
        if len(P) == max_list:
            with open('P_%d.pkl' %(list_num), 'wb') as outfile:
                pickle.dump(P, outfile, pickle.HIGHEST_PROTOCOL)
            P = []
            list_num +=1
             
    p = build_transition_matrix([upper_boundary_condition-1],m,
                                G[len(G)-1][1], z, arrival_rate, upper_boundary_condition, flag = 1)
    P.append(p)
    with open('P_%d.pkl' %(list_num), 'wb') as outfile:
        pickle.dump(P, outfile, pickle.HIGHEST_PROTOCOL)
    del P
    

    return list_num +1
    

In [85]:
def Calculate_taboo_probabilities(P_num):
    W_start = []
    for i in range(P_num):
        with open('P_%d.pkl' %(i), 'rb') as infile:
            P = pickle.load(infile)
        if i == 0:
            w_left = P[0]
        else:
            w_left = np.matmul(w_left, P[0])
        W_start.append(w_left)
        for j in range(1,len(P)):
            w_left = np.matmul(w_left, P[j])
            W_start.append(w_left)
        with open('TP_1_%d.pkl' %(i), 'wb') as outfile:
            pickle.dump(W_start, outfile, pickle.HIGHEST_PROTOCOL)
        W_start = []
        
    W_end = []    
    w_right = P[len(P)-1]
    W_end.append(w_right)
    TP_m_num = P_num
    if len(P) > 1:
        for j in range(len(P)-2,0,-1):
            w_right = np.matmul(P[j], w_right)
            W_end.append(w_right)
        W_end.reverse()
        with open('TP_m_%d.pkl' %(P_num-1), 'wb') as outfile:
                pickle.dump(W_end, outfile, pickle.HIGHEST_PROTOCOL)
        W_end =[]
        w_right = np.matmul(P[0], w_right)
        W_end.append(w_right)
    else:
        TP_m_num = TP_m_num -1
        os.remove('TP_1_%d.pkl' %(P_num-1))
    os.remove('P_%d.pkl' %(P_num-1))    
    for i in range(P_num-2,-1,-1):
        with open('P_%d.pkl' %(i), 'rb') as infile:
            P = pickle.load(infile)
        for j in range(len(P)-1,0,-1):
            w_right = np.matmul(P[j], w_right)
            W_end.append(w_right)
        W_end.reverse()
        with open('TP_m_%d.pkl' %(i), 'wb') as outfile:
            pickle.dump(W_end, outfile, pickle.HIGHEST_PROTOCOL)
        W_end =[]
        w_right = np.matmul(P[0], w_right)
        W_end.append(w_right)
        os.remove('P_%d.pkl' %(i))
        
        
    return TP_m_num

 

In [131]:
def Calculating_distribution(TP_m_num, z, m, upper_boundary_condition ):
    df = pd.DataFrame(z,columns = ['time'])
    df.reset_index(inplace=True)
    for i in range(1,upper_boundary_condition+2):
        df['cumulative_enqueue_%d'%(i)] = 0.0
    for i in range(0,upper_boundary_condition+2):
        df['queue_length_%d'%(i)] = 0.0
    t = 1
    for page in range(TP_m_num):
        with open('TP_1_%d.pkl' %(page), 'rb') as infile:
            TP_1 = pickle.load(infile)
        with open('TP_m_%d.pkl' %(page), 'rb') as infile:
            TP_m = pickle.load(infile)
        for k in range(len(TP_m)):
            for j in range(t,upper_boundary_condition): #  possible values for enqueues
                px = ((TP_1[k][0,j-t])*(TP_m[k][j-t,0]))/(np.dot(TP_1[k],TP_m[k]))
                df.at[t, 'cumulative_enqueue_%d'%(j+1)] = px
                df.at[t, 'queue_length_%d'%(j-t)] = px
            t = t+1
    for page in range(TP_m_num):
        os.remove('TP_1_%d.pkl' %(page))
    for page in range(TP_m_num):
        os.remove('TP_m_%d.pkl' %(page))
    df.at[0, 'cumulative_enqueue_%d'%(1)] = 1
    df.at[m, 'cumulative_enqueue_%d'%(upper_boundary_condition + 1)] = 1
    df.at[m, 'queue_length_%d' %(upper_boundary_condition+1-m)] = 1
    df.at[0, 'queue_length_0'] = 1
    #reduce 0 columns
    bound = upper_boundary_condition +1
    for i in range(upper_boundary_condition+1, 0 , -1):
        if np.all(df['queue_length_%d'%(i)] == 0.0):
            df.drop(['queue_length_%d'%(i)], axis=1)
            bound -= 1
        else:
            break
 
    # Caculate mean         
    df['mean'] = 0.0
    for j in range(bound+1):
        df['mean'] += j*df['queue_length_%d'%(j)]
    
    # Caculate S.D.
    k = df["mean"].tolist()
    k = np.asarray(k)
    x = []
    for i in range(len(df)):
        y = 0.0
        for j in range(bound+1):
            y += math.pow((j-k[i]),2)*df.at[i,'queue_length_%d'%(j)]
        x.append(y)
    x = np.asarray(x)
    x = np.sqrt(x)
    df['sd'] = x
    
    # Caculate median 
    x = []
    z = []
    for i in range(len(df)):
        y = 0.0
        flag = 0
        for j in range(bound+1):
            y += df.at[i,'queue_length_%d'%(j)]
            if y >= 0.5 and flag == 0:
                x.append(j)
                flag = 1
            if y >= 0.9:
                z.append(j)
                break
    df['median'] = x
    df['90_quantile'] = z
    
    # Caculate mode
    x = []
    for i in range(len(df)):
        y = 0.0
        k = 0
        for j in range(bound+1):
            if df.at[i,'queue_length_%d'%(j)] > y:
                y = df.at[i,'queue_length_%d'%(j)] 
                k = j
        x.append(k)
    df['mode'] = x
    
    return df

In [161]:
# This function add all the service timestamps occures after last arrival
def add_reminder_times(result, z_ex, m, upper_boundary_condition, flag ):
    extend = pd.DataFrame(z_ex, columns =['time']) 
    print(len(extend))
    x = []
    for i in range(1,upper_boundary_condition+1):
        extend['cumulative_enqueue_%d'%(i)] = 0.0
    for i in range(0,upper_boundary_condition+2):
        extend['queue_length_%d'%(i)] = 0.0
    extend['cumulative_enqueue_%d'%(upper_boundary_condition + 1)] = 1.0
    for k in range(len(extend)):
        extend.at[k, 'queue_length_%d' %(upper_boundary_condition+1-m-k-1)] = 1
        x.append(upper_boundary_condition+1-m-k-1)
        
    extend['mean'] = x
    extend['sd'] = 0.0
    extend['median'] = x
    extend['90_quantile'] = x
    extend['mode'] = x
    
    result_1 = result
    if flag == 1:
        result_1 = result_1[:-1]
    result_1 = result_1.drop(['index'], axis=1)
    
    frames = [result_1, extend]
    result_2 = pd.concat(frames,  ignore_index = True)
    
    return result_2

In [162]:
def add_labels(full_df, max_enqueue, result, m):
    right = full_df
    right = right.sort_values(by=['order'])
    right = right[['order','queue_length', 'wt', 'arrival']]
    min_order = right['order'].min()
    right['order'] = right['order'] - min_order
    right = right.rename(columns={'order': 'index'})
    result = pd.merge(result, right, on=['index', 'index'])
    return result

The implementaion:

In [165]:
# Read CSV reqired at least sortes 'service' column 
df = pd.read_csv('simulations/busy_periods/bp_%d.csv' %(j)) 
enqueue_vec = df['enqueue'].to_numpy()
enqueue_vec = np.sort(enqueue_vec)
df['inter_arr'] = 0
for i in range(1,len(df)):
    df.at[i,'inter_arr'] = enqueue_vec[i]-enqueue_vec[i-1]
arrival_rate = float(1/np.mean(df['inter_arr'])) #requiered
                
max_list = 500 # size of pickles files for calculations 

# Relevant information to the Algorithm 
max_enqueue = df['enqueue'].max() #requiered
# Boundary conditions
l = len(df) 
upper_boundary_condition = l - 1 # index start from 0, for the (l-1) customer need to subtract 1

# Index continuity
z, flag = build_index_continuity(df , max_enqueue )
z_ex = build_ex_times(df , max_enqueue)
        
m = len(z) - 1  # index start from 0, for the (m) customer need to subtract 1

# Admissible (non-taboo) states
G = build_admissible_states(m , upper_boundary_condition )
        
# (1) Define transition matrices between admissible states
P_num = build_transition_matrices(G, z, m, upper_boundary_condition, arrival_rate, max_list)
        
# (2) Calculating the matrices of taboo probabilities
TP_m_num = Calculate_taboo_probabilities(P_num) 
        
# (3) Calculating the distribution for cumulative number of arrivals
result = Calculating_distribution(TP_m_num , z, m, upper_boundary_condition )
result_1 = add_reminder_times(result, z_ex, m, upper_boundary_condition, flag ) 
result_1.reset_index(inplace=True)
result_1 = add_labels(df, max_enqueue, result_1, m )
        

#Save result
result_1.to_csv(r'QIE_outputs/res_%s.csv' %(j), index = False) 
